**Table of contents**<a id='toc0_'></a>    
- [Inferring human DFE using three commands](#toc1_)    
- [Command2: Build a reference spectra cache](#toc2_)    
  - [Local example](#toc2_1_)    
  - [Submit a hoffman2 job](#toc2_2_)    
    - [Log output](#toc2_2_1_)    
    - [File output](#toc2_2_2_)    
- [Command3: Run DFE inference assuming a gamma distributed DFE](#toc3_)    
  - [Local example](#toc3_1_)    
    - [Log output](#toc3_1_1_)    
    - [File output](#toc3_1_2_)    
    - [What is a good enough DFE inference?](#toc3_1_3_)    
- [Command3.a: Run DFE inference assuming a different DFE function](#toc4_)    
  - [Local example](#toc4_1_)    
    - [Output of the lognormal model and compare with the gamma model](#toc4_1_1_)    
- [About](#toc5_)    
  - [Disclaimer](#toc5_1_)    
  - [Citation](#toc5_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Inferring human DFE using three commands](#toc0_)

**Part2: dfe inference**

Date: 2023-03-10 12:38:44

Author: Meixi Lin

# <a id='toc2_'></a>[Command2: Build a reference spectra cache](#toc0_)

The script is [DFE1D_refspectra](../../workflow/DFE/DFE1D_refspectra.py)

In this step, we calculate the expected SFS for each selection coefficient possible `s` and this cache will be used to infer the DFE in the final DFE inference step. This step takes longer and more computational resources. But you only need to run it once. 

**HIGHLY RECOMMEND: Run this in a server**

## <a id='toc2_1_'></a>[Local example](#toc0_)

In [1]:
%%bash
WORKSCRIPT='../../workflow/DFE/DFE1D_refspectra.py'
python $WORKSCRIPT -h

usage: DFE1D_refspectra.py [-h] demog_model demog_params ns outprefix

Generate a demographics informed precomputed spectra for each
species/population.

positional arguments:
  demog_model   Demographic model to use.
  demog_params  Demographic parameters for demog+sel models. Please provide in
                the forms of `nu`,`T`. must be "," delimited. Make sure the
                values are in the correct orders (e.g."1.89,0.29" for 2Epoch
                or "1.8,1.2,0.32,0.28" for 3Epoch
  ns            Number of samples in the SFS to generate
  outprefix     Path/NamePrefix to the output file

options:
  -h, --help    show this help message and exit



```bash
python $WORKSCRIPT \
'two_epoch' '2.332027,0.42853' '100' './output/dfe/refspectra/HS100' \
&> './output/logs/dfe_refspectra_human.log'
```

## <a id='toc2_2_'></a>[Submit a hoffman2 job](#toc0_)


```bash
cd /u/project/klohmuel/meixilin/finwhale_DFE/scripts_varDFE/varDFE/example/human_dfe
qsub human_dfe_refspectra.sh
```

### <a id='toc2_2_1_'></a>[Log output](#toc0_)

The log generated from the reference spectra contains the following information:

1. Input parameter parsed

```
INFO:[2023-03-10 16:16:22] - Beginning execution of ../../workflow/DFE/DFE1D_refspectra.py in directory /u/project/klohmuel/meixilin/finwhale_DFE/scripts_varDFE/varDFE/example/human_dfe
INFO:Parsed the following arguments:
	demog_model = two_epoch
	demog_params = 2.332027,0.42853
	ns = 100
	outprefix = ./output/dfe/refspectra/HS100
```

2. The `s` values used for the reference spectra

```
6: -8709.635899560806
62: -2398.83291901949
98: -1047.1285480508996
127: -537.0317963702527
148: -331.1311214825911
304: -9.120108393559097
...
1015: 0.00013803842646028852
1049: 0.0003019951720402016
1085: 0.0006918309709189362
1121: 0.001584893192461114
1156: 0.0035481338923357567
1190: 0.00776247116628692
1224: 0.016982436524617443
```

3. Summary of the output

```
INFO:[2023-03-10 16:20:09] - Number of negative gammas: (901,). Number of all gammas: (1602,)
INFO:[2023-03-10 16:20:13] - SUCCESS! Spectra saved to ./output/dfe/refspectra/HS100_DFESpectrum.bpkl
```

By default, both negative and positive `s` are used.

Notes:

You are going to get warnings for the very negative `s` (very deleterious). 
It's caused by `dadi.Numerics.make_extrap_func` returning tiny negative values in some SFS entries due to floating point rounding errors.
You can manually go in and fill in the negative values but my tests showed that it does not impact the results and can be safely ignored.
**HOWEVER**, if you observe `Inf` values in the reference spectra generated, take caution and resolve it before proceeding to DFE inference. 
For detailed explanation, see [fitdadi_manual](https://github.com/LohmuellerLab/fitdadi/blob/master/manual_examples/manual.pdf) (end of pg.3).

```
WARNING:Numerics:Extrapolation may have failed. Check resulting frequency spectrum for unexpected results.
WARNING:Numerics:Extrapolation may have failed. Check resulting frequency spectrum for unexpected results.
WARNING:Numerics:Extrapolation may have failed. Check resulting frequency spectrum for unexpected results.
WARNING:Numerics:Extrapolation may have failed. Check resulting frequency spectrum for unexpected results.
```


### <a id='toc2_2_2_'></a>[File output](#toc0_)

This step outputs two files

```
.
└── refspectra
    ├── HS100_DFESpectrum.bpkl......... the stored reference spectra 
    ├── HS100_DFESpectrum_QC.pdf....... QC plots showing the SFS under various different `s`
```

[HS100_DFESpectrum_QC](./output/dfe/refspectra/HS100_DFESpectrum_QC.pdf)

In [2]:
# load the pickled file
import pickle
ref_spectra=pickle.load(open('./output/dfe/refspectra/HS100_DFESpectrum.bpkl','rb'))

# some properties of the reference spectra
print(ref_spectra.demo_sel_func) # the demographic function used
print(ref_spectra.ns) # number of samples (chromosomes)

<function two_epoch at 0x1584957e0>
[100]


In [3]:
# note the very small negative SFS values in the very negative selection coefficient
print(ref_spectra.gammas[0])
ref_spectra.spectra[0] #  (e.g. -1.82844415e-139)

-10000.0


array([ 8.36880282e+000,  4.99037949e-003,  5.29502462e-006,
        7.42633474e-009,  1.15707606e-011,  1.90280544e-014,
        3.22726362e-017,  5.57352818e-020,  9.72626245e-023,
        1.70657367e-025,  3.00045670e-028,  5.27313469e-031,
        9.24644970e-034,  1.61544911e-036,  2.80891128e-039,
        4.85639704e-042,  8.34243104e-045,  1.42296002e-047,
        2.40862777e-050,  4.04394765e-053,  6.73134312e-056,
        1.11038622e-058,  1.81446399e-061,  2.93600822e-064,
        4.70259713e-067,  7.45296739e-070,  1.16834699e-072,
        1.81094672e-075,  2.77440430e-078,  4.19953880e-081,
        6.27822542e-084,  9.26636401e-087,  1.34974153e-089,
        1.93950250e-092,  2.74825086e-095,  3.83861707e-098,
        5.28287240e-101,  7.16086469e-104,  9.55614271e-107,
        1.25499553e-109,  1.62130056e-112,  2.05951475e-115,
        2.57136242e-118,  3.15409189e-121,  3.79937541e-124,
        4.49251302e-127,  5.21214217e-130,  5.93063452e-133,
        6.61645073e-136,

In [4]:
# note the excess of common alleles in the very positive selection coefficient
print(ref_spectra.gammas[-1])
ref_spectra.spectra[-1] 

100.0


array([22.63730294,  2.35556257,  1.18980054,  0.80137769,  0.60729403,
        0.49094927,  0.41347678,  0.35821951,  0.31684905,  0.28473858,
        0.25911211,  0.23820316,  0.22083418,  0.20619001,  0.19368845,
        0.18290266,  0.17351257,  0.16527349,  0.15799518,  0.15152754,
        0.14575056,  0.14056714,  0.13589795,  0.13167752,  0.12785136,
        0.12437381,  0.12120628,  0.11831602,  0.11567504,  0.11325929,
        0.11104804,  0.10902332,  0.10716952,  0.10547304,  0.10392196,
        0.10250588,  0.10121566,  0.10004328,  0.0989817 ,  0.09802475,
        0.09716703,  0.09640384,  0.09573107,  0.0951452 ,  0.09464321,
        0.09422258,  0.09388119,  0.09361737,  0.09342984,  0.09331768,
        0.09328035,  0.09331768,  0.09342984,  0.09361737,  0.09388119,
        0.09422258,  0.09464321,  0.0951452 ,  0.09573107,  0.09640384,
        0.09716703,  0.09802475,  0.0989817 ,  0.10004328,  0.10121566,
        0.10250588,  0.10392196,  0.10547304,  0.10716952,  0.10

# <a id='toc3_'></a>[Command3: Run DFE inference assuming a gamma distributed DFE](#toc0_)

The script is [DFE1D_inferenceFIM](../../workflow/DFE/DFE1D_inferenceFIM.py)

## <a id='toc3_1_'></a>[Local example](#toc0_)

In [5]:
%%bash
WORKSCRIPT='../../workflow/DFE/DFE1D_inferenceFIM.py'

python $WORKSCRIPT -h

python $WORKSCRIPT \
--pop 'HS100' --mu '2.50E-08' --Lcds '19089129' --NS_S_scaling '2.31' --Nrun 5 \
'sfs/MIS-HS100.sfs' './output/dfe/refspectra/HS100_DFESpectrum.bpkl' 'gamma' '4061.641015' './output/dfe/gamma' &> './output/logs/dfe_human_gamma.log'

usage: DFE1D_inferenceFIM.py [-h] --pop POP --mu MU --Lcds LCDS --NS_S_scaling
                             NS_S_SCALING [--Nrun NRUN] [--mask_singleton]
                             sfs ref_spectra pdfname theta_syn outdir

Run DFE inference from precomputed spectra for each species/population.

positional arguments:
  sfs                   path to FOLDED NONSYN SFS in dadi format from easysfs
                        (mask optional)
  ref_spectra           path to reference DFE spectra
  pdfname               DFE functional form to use.
  theta_syn             Theta of synonymous regions from demographic
                        inference.
  outdir                path to output directory

options:
  -h, --help            show this help message and exit
  --pop POP             population identifier, e.g. 'HS100'
  --mu MU               supply exon mutation rate in mutation/bp/gen
  --Lcds LCDS           number of called CDS sites that went into making SFS
                        (monomo

### <a id='toc3_1_1_'></a>[Log output](#toc0_)

This script gives (hopefully) all the information you need for DFE inference and comparisons: 

In the output log, it provides: 

1. Input parameters parsed: 

```
INFO:[2023-03-10 17:01:36] - Beginning execution of ../../workflow/DFE/DFE1D_inferenceFIM.py in directory /Users/linmeixi/Lab/finwhale_DFE/scripts_varDFE/varDFE/example/human_dfe
INFO:Parsed the following arguments:
	pop = HS100
	mu = 2.5e-08
	Lcds = 19089129.0
	NS_S_scaling = 2.31
	Nrun = 5
	mask_singleton = False
	sfs = sfs/MIS-HS100.sfs
	ref_spectra = ./output/dfe/refspectra/HS100_DFESpectrum.bpkl
	pdfname = gamma
	theta_syn = 4061.641015
	outdir = ./output/dfe/gamma
	Lsyn = 5767108.0
	Lnonsyn = 13322021.0
	theta_nonsyn = 9382.39074465
	Nanc = 7042.769122756155
```

2. Each iteration's log-likelihood and parameter values 

```
INFO:[2023-03-10 17:01:36] - Beginning DFE optimization dadi.Inference.optimize_log assuming PDF <function gamma at 0x1140a3f40>. Total runs = 5.
	params=['shape', 'scale']
	upper_bound = [2.0, 1000000.0]
	lower_bound = [0.001, 0.01]
	initial_val = [0.2, 4000.0]
5       , -2287.85    , array([ 0.0917182  ,  4222.75    ])
10      , -2294.93    , array([ 0.0916306  ,  4221.62    ])
15      , -2293.71    , array([ 0.0916526  ,  4219.75    ])
20      , -2287.7     , array([ 0.0917453  ,  4215.25    ])
25      , -17738.8    , array([ 0.562842   ,  0.0763085  ])
...
145     , -241.872    , array([ 0.188688   ,  1045.17    ])
150     , -241.872    , array([ 0.188688   ,  1046.22    ])
155     , -241.872    , array([ 0.188688   ,  1045.18    ])
160     , -241.872    , array([ 0.188688   ,  1045.17    ])
165     , -241.872    , array([ 0.188688   ,  1045.17    ])
INFO:[2023-03-10 17:01:49] - Rep00. Output *_unfolded.expSFS, *_folded.expSFS, *.png, *.txt to ./output/dfe/gamma/detail_5runs/HS100_DFE_gamma_run00
```

3. Top three runs with the best log-likelihoods

```
INFO:[2023-03-10 17:02:32] - Top 3 runs:
  runNum   rundate  Nrun  maxiter    pop                sfs                                     ref_spectra  mask_singleton   ns pdf_func integrate_func                optimize_func            mu        Lcds  NS_S     Lnonsyn    upper_bound lower_bound     initval                              initval_p0  theta_nonsyn    ll_model     ll_data         Nanc     shape        scale  shape_us  scale_us
0     02  20230310     5      100  HS100  sfs/MIS-HS100.sfs  ./output/dfe/refspectra/HS100_DFESpectrum.bpkl           False  100    gamma      integrate  dadi.Inference.optimize_log  2.500000e-08  19089129.0  2.31  13322021.0  2.0,1000000.0  0.001,0.01  0.2,4000.0  0.12932708502604104,3045.9212717865457   9382.390745 -241.852408 -183.692993  7042.769123  0.189191  1030.903026  0.189191  0.073189
1     03  20230310     5      100  HS100  sfs/MIS-HS100.sfs  ./output/dfe/refspectra/HS100_DFESpectrum.bpkl           False  100    gamma      integrate  dadi.Inference.optimize_log  2.500000e-08  19089129.0  2.31  13322021.0  2.0,1000000.0  0.001,0.01  0.2,4000.0   0.10152128485868697,6842.350108617073   9382.390745 -241.871329 -183.692993  7042.769123  0.188697  1044.854182  0.188697  0.074179
2     00  20230310     5      100  HS100  sfs/MIS-HS100.sfs  ./output/dfe/refspectra/HS100_DFESpectrum.bpkl           False  100    gamma      integrate  dadi.Inference.optimize_log  2.500000e-08  19089129.0  2.31  13322021.0  2.0,1000000.0  0.001,0.01  0.2,4000.0   0.24829555894494987,4966.103022862596   9382.390745 -241.871749 -183.692993  7042.769123  0.188688  1045.172032  0.188688  0.074202
```

In this example, the best run was: `shape = 0.189191, scale = 0.073189`

4. Convergence of each replicated runs. 

```
INFO:[2023-03-10 17:02:32] - Convergence of parameters:
{'ll_model': 0.0193586167822275, 'shape': 0.0010620159816266369, 'scale': 0.005451104835427154}
```

Convergence is calculated using `varDFE.Misc.Util.CheckConvergence()` function from the top 20 runs with the best log-likelihood. 

* `ll_model`'s convergence defined as the differences between the log-likelihood of the best run and the 20th best run. 
* `params`'s convergence defined as the coefficient of variation in the 20 best runs. 

5. Fisher's Information Matrix (FIM) based standard deviation estimates. 

```
INFO:[2023-03-10 17:02:34] - Best params STDEV = [4.71814774e-03 1.23428219e+02]
```


### <a id='toc3_1_2_'></a>[File output](#toc0_)

In the output folders, the files are organized as following: 

```
gamma/
├── HS100_DFE_gamma_PDF.pdf
├── HS100_DFE_gamma_SFS.pdf
├── HS100_DFE_gamma_summary.txt
├── bestrun
│   ├── HS100_DFE_gamma_run02.SD.txt
│   ├── HS100_DFE_gamma_run02.info.txt
│   ├── HS100_DFE_gamma_run02.png
│   ├── HS100_DFE_gamma_run02.txt
│   ├── HS100_DFE_gamma_run02_folded.expSFS
│   └── HS100_DFE_gamma_run02_unfolded.expSFS
└── detail_5runs
```

* `HS100_DFE_gamma_SFS.pdf`: The SFS fit for the best model
* `HS100_DFE_gamma_PDF.pdf`: The probability density function (PDF) for the best model
* `HS100_DFE_gamma_summary.txt`: Tabulated data for all the runs
* `bestrun/`: the folder with all the information for the best run
    * `HS100_DFE_gamma_run02.info.txt`: Tabulated data for the best run with uncertainty estimate appended (FIM + convergence). 
    * `HS100_DFE_gamma_run02.png`: `dadi`'s original plot. diagonostic for looking at the residuals. 
    * `HS100_DFE_gamma_run02_folded.expSFS`: folded expected SFS from this run. 
 

### <a id='toc3_1_3_'></a>[What is a good enough DFE inference?](#toc0_)


Disclaimer: this is based on personal experiences. 

1. Check the convergence of the model. 

In [6]:
import pandas as pd
import os
files = os.listdir('./output/dfe/gamma/bestrun/')
resfile = [ii for ii in files if '.info.txt' in ii][0]
resdt = pd.read_csv('./output/dfe/gamma/bestrun/'+resfile, sep = '\t')


2. Check if SD estimation is available and not very large. 

In [7]:
resdt.loc[:, resdt.columns.str.endswith('_sd')]

,shape_sd,scale_sd
0,0.004718,123.428219



3. Visually check the plot of model-data SFS. 
    * dadi output

    ![dadi_modelfit](output/dfe/gamma/bestrun/HS100_DFE_gamma_run02.png)
    
    * sometimes barplots are more helpful
    [varDFE_barplot](output/dfe/gamma/HS100_DFE_gamma_SFS.pdf)

The DFE inference is reasonable. 

# <a id='toc4_'></a>[Command3.a: Run DFE inference assuming a different DFE function](#toc0_)

Currently, this pipeline supports the following DFE functions: 

1. gamma: Gamma distribution
2. neugamma: Gamma distribution + neutral point mass 
3. gammalet: Gamma distribution + lethal mutations
4. neugammalet: Gamma distribution + neutral point mass + lethal mutations
5. lognormal: Log-normal distribution
6. lourenco_eq: Fisher's Geometric Model based distribution (derived in Lourenço et al. 2011)


Here we try to run the lognormal distribution too.

## <a id='toc4_1_'></a>[Local example](#toc0_)

In [8]:
%%bash
WORKSCRIPT='../../workflow/DFE/DFE1D_inferenceFIM.py'

# here we only need to change gamma to lognormal and everything else are taken care of
python $WORKSCRIPT \
--pop 'HS100' --mu '2.50E-08' --Lcds '19089129' --NS_S_scaling '2.31' --Nrun 5 \
'sfs/MIS-HS100.sfs' './output/dfe/refspectra/HS100_DFESpectrum.bpkl' 'lognormal' '4061.641015' './output/dfe/lognormal' &> './output/logs/dfe_human_lognormal.log'

### <a id='toc4_1_1_'></a>[Output of the lognormal model and compare with the gamma model](#toc0_)

In [9]:
files = os.listdir('./output/dfe/lognormal/bestrun/')
resfilel = [ii for ii in files if '.info.txt' in ii][0]
resdtl = pd.read_csv('./output/dfe/lognormal/bestrun/'+resfilel, sep = '\t')

In [10]:
# gamma distribution
resdt[['pdf_func','ll_model', 'll_data', 'shape', 'scale', 'shape_sd', 'scale_sd']]

,pdf_func,ll_model,ll_data,shape,scale,shape_sd,scale_sd
0,gamma,-241.852408,-183.692993,0.189191,1030.903026,0.004718,123.428219


In [11]:
# lognormal distribution
resdtl[['pdf_func','ll_model', 'll_data', 'mus', 'sigma', 'mus_sd', 'sigma_sd']]

,pdf_func,ll_model,ll_data,mus,sigma,mus_sd,sigma_sd
0,lognormal,-273.925776,-183.692993,2.694213,4.857347,0.042779,0.124465


Both `lognormal` and `gamma` distribution had reasonably good fit but the gamma distribution performed better with a higher log-likelihood. 

# <a id='toc5_'></a>[About](#toc0_)

## <a id='toc5_1_'></a>[Disclaimer](#toc0_)

`varDFE` and this tutorial are provided "as is" without any warranties or representations of any kind, express or implied. I make no guarantees or warranties regarding the accuracy, reliability, completeness, suitability, or timeliness of the software.

## <a id='toc5_2_'></a>[Citation](#toc0_)

Remember to cite the `dadi` package and `fitdadi` this package is based on as well. 

```
RN Gutenkunst, RD Hernandez, SH Williamson, CD Bustamante "Inferring the joint demographic history of multiple populations from multidimensional SNP data" PLoS Genetics 5:e1000695 (2009).

BY Kim, CD Huber, KE Lohmueller "Inference of the Distribution of Selection Coefficients for New Nonsynonymous Mutations Using Large Samples" Genetics 206:345 (2017).
```

